<a href="https://colab.research.google.com/github/david06baez/Trabajo_Final1/blob/main/PARKFLOW.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PARKFLOW
## INTEGRANTES:
- DAVID FERNANDO BÁEZ
- JUAN PABLO MARÍN
- SANTIAGO OSORIO


## LIBRERÍAS:
- CSV: Esta librería nos permite leer y escribir archivos en CSV. Es útil para almacenar y manipular datos estructurados de manera sencilla, como registros de entradas y salidas. En nuestro trabajo la usamos para generar un archivo con el historial diario del parqueadero, guardando información como el nombre, placa, hora de ingreso, hora de salida y el total pagado.
- Datetime: Es un módulo de Python que permite trabajar con fechas y horas. Proporciona funciones para obtener la fecha y hora actual, calcular diferencias entre tiempos. En el proyecto la usamos para registrar la hora de ingreso y salida de los vehículos, así como para calcular el tiempo total de estancia en el parqueadero.
- Pandas: Es una librería muy útil para el análisis y manipulación de datos. Facilita el trabajo con grandes volúmenes de información en estructuras tipo tabla. Fue empleada en el proyecto para leer el archivo csv generado al final del día con los registros de ingreso y salida de vehículos.

In [1]:
#Importamos librerías
import csv
from datetime import datetime, timedelta
import pandas as pd

## Almacenamiento de datos
Para guardar la información mientras el programa se ejecute, usamos listas y diccionarios. Con las listas manejamos cosas como los usuarios registrados, los carros que están dentro del parqueadero, los espacios disponibles y el historial del día. Cada elemento en esas listas significa un registro. También usamos diccionarios para datos más organizados, por ejemplo, para guardar la información de cada vehículo que entra o sale, o para las credenciales del administrador.

In [2]:
#Almacenamos los datos en las siguientes listas
Usuarios = []
Vehiculos_en_parqueo = []
Historico_vehiculos = []
Espacios_disponibles = [f'P{i+1}' for i in range(50)]  #50 celdas numeradas del P1 al P50, este es el espacio de nuestro parqueadero.
Credenciales_admin = {'admin': 'admin123'}  #Usuario y contraseña del admin

## Validaciones de los datos
Para asegurar que los datos ingresados por los usuarios sean correctos y consistentes, el sistema implementa una serie de funciones que realizan validaciones básicas antes de guardar cualquier información. Estas funciones devuelven una lista con los errores encontrados, lo que permite informar al usuario de forma clara qué debe corregir.

def validar_nombre(nombre):

Esta función se encarga de validar el nombre del usuario. Revisa que el nombre tenga al menos 3 caracteres y que no contenga números, ya que un nombre válido solo debe contener letras. Si no cumple con estas condiciones, devuelve los mensajes de error correspondientes.

def validar_apellido(apellido):

Funciona de manera similar a la anterior, pero aplicada al apellido. Se asegura de que el apellido tenga una longitud mínima de 3 letras y que no incluya dígitos. Esto ayuda a mantener una base de datos limpia y coherente.

def validar_documento(documento):

Esta validación garantiza que el documento de identidad ingresado sea numérico y que su longitud esté entre 3 y 15 caracteres. De esta forma, se evita el ingreso de documentos inválidos o extremadamente cortos o largos.

def validar_placa(placa):

Valida que la placa del vehículo tenga exactamente 6 caracteres, en un formato específico: 3 letras seguidas de 3 número. Esto permite mantener un registro estandarizado de las placas y evitar errores de escritura o formatos incorrectos.

Todas estas validaciones se ejecutan al momento de registrar un nuevo usuario. Si alguna de ellas falla, el sistema muestra los errores y no permite avanzar hasta que la información sea válida.

In [3]:
#Validaciones de los datos ingresados por el usuario
def validar_nombre(nombre):
    '''
    Recibe un string y devuelve una lista con los errores encontrados.
    '''
    errores = []
    if len(nombre) < 3:
        errores.append("El nombre debe tener al menos 3 letras.")
    if any(char.isdigit() for char in nombre):
        errores.append("El nombre no puede contener números.")
    return errores

def validar_apellido(apellido):
    '''
    Recibe un string y devuelve una lista con los errores encontrados.
    '''
    errores = []
    if len(apellido) < 3:
        errores.append("El apellido debe tener al menos 3 letras.")
    if any(char.isdigit() for char in apellido):
        errores.append("El apellido no puede contener números.")
    return errores

def validar_documento(documento):
    '''
    Recibe un string y devuelve una lista con los errores encontrados.
    '''
    errores = []
    if not documento.isdigit():
        errores.append("El documento debe contener solo números.")
    if not (3 <= len(documento) <= 15):
        errores.append("El documento debe tener entre 3 y 15 dígitos.")
    return errores

def validar_placa(placa):
    '''
    Recibe un string y devuelve una lista con los errores encontrados.
    '''
    errores = []
    if len(placa) != 6:
        errores.append("La placa debe tener exactamente 6 caracteres.")
    if not (placa[:3].isalpha() and placa[3:].isdigit()):
        errores.append("La placa debe tener 3 letras seguidas de 3 números (Ej: ABC123).")
    return errores

## Código principal
Es el punto de entrada de nuestro sistema, acá activamos cada parte del programa según la opción que el usuario seleccione.

In [4]:
#Mensaje de bienvenida
print('-' * 74)
print("""
  ____  _                           _     _
 |  _ \(_)                         (_)   | |
 | |_) |_  ___ _ ____   _____ _ __  _  __| | ___  ___    __ _
 |  _ <| |/ _ \ '_ \ \ / / _ \ '_ \| |/ _` |/ _ \/ __|  / _` |
 | |_) | |  __/ | | \ V /  __/ | | | | (_| | (_) \__ \ | (_| |
 |____/|_|\___|_| |_|\_/ \___|_| |_|_|\__,_|\___/|___/  \__,_|
  _                                   _____           _    _
 | |                                 |  __ \         | |  (_)
 | |    _   ___  ___   _ _ __ _   _  | |__) |_ _ _ __| | ___ _ __   __ _
 | |   | | | \ \/ / | | | '__| | | | |  ___/ _` | '__| |/ / | '_ \ / _` |
 | |___| |_| |>  <| |_| | |  | |_| | | |  | (_| | |  |   <| | | | | (_| |
 |______\__,_/_/\_\__, _|_|   \__, | |_|   \__,_|_|  |_|\_\_|_| |_|\__, |
                               __/ |                                __/ |
                              |___/                                |___/
""")
print('-' * 74)

#Bucle principal
while True:
    print('*' * 3, 'MENÚ DE OPCIONES', '*' * 3)
    print('1. Registrar usuario.')
    print('2. Registrar ingreso de vehículo.')
    print('3. Retirar vehículo y calcular el costo.')
    print('4. Ingreso como administrador.')
    print('5. Cerrar día y generar CSV.')
    print('6. Salir del programa.')

    opcion_input = input('Ingrese la opción deseada: ')
    if not opcion_input.isdigit():
        print("Debe ingresar un número.")
        continue
    opcion = int(opcion_input)

    if opcion == 1:
        print('Bienvenido al sistema de registro de usuario.')
        errores_totales = []

        nombre = input('Ingrese su nombre: ').capitalize()
        errores_totales += validar_nombre(nombre)

        apellido = input('Ingrese su apellido: ').capitalize()
        errores_totales += validar_apellido(apellido)

        documento = input('Ingrese su documento de identidad: ')
        errores_totales += validar_documento(documento)

        placa = input('Ingrese la placa de su vehículo: ').upper()
        errores_totales += validar_placa(placa)

        if errores_totales:
            print('Se encontraron los siguientes errores:')
            for err in errores_totales:
                print("-", err)
        else:
            Usuarios.append([nombre, apellido, documento, placa])
            print(f'{nombre} {apellido}, su usuario se ha registrado con éxito.')

    elif opcion == 2:
        print('Bienvenido al sistema de ingreso de vehículos.')
        if not Espacios_disponibles:
            print('El parqueadero está lleno.')
            continue

        placa_ingresar = input('Ingrese la placa del vehículo: ').upper()
        usuario_encontrado = None
        for usuario in Usuarios:
            if usuario[3] == placa_ingresar:
                usuario_encontrado = usuario
                break

        if usuario_encontrado:
            hora_ingreso = datetime.utcnow() - timedelta(hours=5)
            celda_asignada = Espacios_disponibles.pop(0)
            vehiculo = [usuario_encontrado[0], usuario_encontrado[3], hora_ingreso, celda_asignada]
            Vehiculos_en_parqueo.append(vehiculo)

            registro = {
                "Nombre": usuario_encontrado[0],
                "Apellido": usuario_encontrado[1],
                "Placa": usuario_encontrado[3],
                "Hora_ingreso": hora_ingreso,
                "Hora_salida": None,
                "Total_pagado": None,
                "Celda": celda_asignada
            }
            Historico_vehiculos.append(registro)

            print('Vehículo ingresado correctamente en la celda', celda_asignada)
            print('*'*5,'FACTURA','*'*5)
            print('Hora de ingreso:', hora_ingreso.strftime('%Y-%m-%d %H:%M:%S'))
            print('Usuario:', usuario_encontrado[0])
            print('Placa:', usuario_encontrado[3])
            print('Celda asignada:', celda_asignada)

        else:
            print('El vehículo no se encuentra registrado.')

    elif opcion == 3:
        print('Bienvenido al sistema de retiro de vehículo.')
        placa_retirar = input('Ingrese la placa del vehículo a retirar: ').upper()
        vehiculo_encontrado = None
        for vehiculo in Vehiculos_en_parqueo:
            if vehiculo[1] == placa_retirar:
                vehiculo_encontrado = vehiculo
                break

        if vehiculo_encontrado:
            hora_salida = datetime.utcnow() - timedelta(hours=5)
            hora_ingreso = vehiculo_encontrado[2]
            tiempo_total_min = (hora_salida - hora_ingreso).total_seconds() / 60

            if tiempo_total_min <= 0:
                print("Error: la hora de salida debe ser mayor que la de ingreso.")
            else:
                horas = int(tiempo_total_min) // 60
                minutos_restantes = int(tiempo_total_min) % 60
                cuartos_hora = minutos_restantes // 15

                cobro_horas = horas * 7000
                cobro_cuartos = cuartos_hora * 1500
                total_pagar = max(7000, cobro_horas + cobro_cuartos)

                print(f'Tiempo total: {horas}h {minutos_restantes}min')
                print(f'Total a pagar: ${total_pagar}')

                Vehiculos_en_parqueo.remove(vehiculo_encontrado)
                Espacios_disponibles.append(vehiculo_encontrado[3])  # devolver celda

                for registro in Historico_vehiculos:
                    if registro["Placa"] == placa_retirar and registro["Hora_salida"] is None:
                        registro["Hora_salida"] = hora_salida
                        registro["Total_pagado"] = total_pagar
                        break
                print('Vehículo retirado correctamente.')
        else:
            print('Vehículo no encontrado.')

    elif opcion == 4:
        print('Ingreso como administrador.')
        usuario = input("Usuario: ")
        contraseña = input("Contraseña: ")
        if usuario in Credenciales_admin and Credenciales_admin[usuario] == contraseña:
            print('Acceso concedido.')

            total_registrados = len(Historico_vehiculos)
            total_retirados = sum(1 for r in Historico_vehiculos if r["Hora_salida"] is not None)
            total_en_parqueo = total_registrados - total_retirados
            total_pagado = sum(r["Total_pagado"] for r in Historico_vehiculos if r["Total_pagado"] is not None)

            tiempos = []
            for r in Historico_vehiculos:
                if r["Hora_salida"] is not None:
                    tiempo = (r["Hora_salida"] - r["Hora_ingreso"]).total_seconds() / 60
                    tiempos.append((r["Placa"], tiempo))

            tiempo_prom = sum(t for _, t in tiempos) / len(tiempos) if tiempos else 0
            if tiempos:
                max_tiempo = max(tiempos, key=lambda x: x[1])
                min_tiempo = min(tiempos, key=lambda x: x[1])
            else:
                max_tiempo = min_tiempo = ("N/A", 0)

            print("--- Reporte de Administración ---")
            print(f"Total vehículos registrados: {total_registrados}")
            print(f"Total vehículos retirados: {total_retirados}")
            print(f"Total vehículos en parqueo: {total_en_parqueo}")
            print(f"Total pagado: ${total_pagado}")
            print(f"Tiempo promedio de estancia: {round(tiempo_prom, 2)} minutos")
            print(f"Vehículo con mayor tiempo: {max_tiempo[0]} ({round(max_tiempo[1], 2)} min)")
            print(f"Vehículo con menor tiempo: {min_tiempo[0]} ({round(min_tiempo[1], 2)} min)")
            print("Usuarios registrados:")
            for u in Usuarios:
                print(f"- {u[0]} {u[1]} - Placa: {u[3]}")
            print("Ocupación de celdas:")
            ocupadas = [v[3] for v in Vehiculos_en_parqueo]
            for i in range(50):
                celda = f'P{i+1}'
                estado = "Ocupada" if celda in ocupadas else "Libre"
                print(f"{celda}: {estado}")
        else:
            print("Acceso denegado.")

    elif opcion == 5:
        print("Cerrando el día. Generando archivo CSV...")

        with open('registro_dia.csv', 'w', newline='') as archivo_csv:
            writer = csv.writer(archivo_csv)
            writer.writerow(['Nombre', 'Apellido', 'Placa', 'Hora de Ingreso', 'Hora de Salida', 'Total Pagado', 'Celda'])

            for registro in Historico_vehiculos:
                hora_ingreso = registro["Hora_ingreso"].strftime('%Y-%m-%d %H:%M:%S')
                hora_salida = registro["Hora_salida"].strftime('%Y-%m-%d %H:%M:%S') if registro["Hora_salida"] else "En parqueadero"
                total = registro["Total_pagado"] if registro["Total_pagado"] is not None else "N/A"
                writer.writerow([registro["Nombre"], registro["Apellido"], registro["Placa"], hora_ingreso, hora_salida, total, registro["Celda"]])

        print("Archivo CSV generado como 'registro_dia.csv'. ¡Hasta mañana!")
        break

    elif opcion == 6:
        print("Saliendo del programa. ¡Hasta luego!")
        break

    else:
        print("Debe ingresar una opción válida.")



--------------------------------------------------------------------------

  ____  _                           _     _
 |  _ \(_)                         (_)   | |
 | |_) |_  ___ _ ____   _____ _ __  _  __| | ___  ___    __ _
 |  _ <| |/ _ \ '_ \ \ / / _ \ '_ \| |/ _` |/ _ \/ __|  / _` |
 | |_) | |  __/ | | \ V /  __/ | | | | (_| | (_) \__ \ | (_| |
 |____/|_|\___|_| |_|\_/ \___|_| |_|_|\__,_|\___/|___/  \__,_|
  _                                   _____           _    _
 | |                                 |  __ \         | |  (_)
 | |    _   ___  ___   _ _ __ _   _  | |__) |_ _ _ __| | ___ _ __   __ _
 | |   | | | \ \/ / | | | '__| | | | |  ___/ _` | '__| |/ / | '_ \ / _` |
 | |___| |_| |>  <| |_| | |  | |_| | | |  | (_| | |  |   <| | | | | (_| |
 |______\__,_/_/\_\__, _|_|   \__, | |_|   \__,_|_|  |_|\_\_|_| |_|\__, |
                               __/ |                                __/ |
                              |___/                                |___/

------------------

## Lectura del archivo CSV
Con pd.read_csv('registro_dia.csv') leemos el archivo descargado utilizando la librería pandas. Esto nos permite visualizar y manipular los datos de los vehículos registrados durante el día, como nombres, placas, horas de ingreso y salida, entre otros.

In [ ]:
archivo = pd.read_csv('registro_dia.csv')
archivo

## Descarga del archivo CSV
Usamos files.download('registro_dia.csv') para descargar el archivo CSV generado con los registros del parqueadero. Esta instrucción permite obtener el archivo directamente desde Google Colab hacia nuestro computador, facilitando su almacenamiento o análisis posterior.

In [ ]:
from google.colab import files
files.download('registro_dia.csv')